In [1]:
import pandas as pd 
import numpy as np 
import os
import pickle
import matplotlib.pyplot as plt
import json 
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout,add

In [ ]:
model = load_model("besst_model.h5")

In [ ]:
model_vgg = Model(inputs = model.inputs,outputs = model.layers[-2].output)

In [ ]:
def preprocess_image(img):
    #load the image from file 
    img = load_img(img,target_size = (224,224))
    #image pixel to numpy array 
    img = img_to_array(img)
    #reshapre data for model
    img = np.expand_dims(img,axis=0)
    #preprocess image for vgg
    img = preprocess_input(img)
    return img

In [ ]:
def encode_image(img):
    img = preprocess_image(img)
    feature_vector = model_vgg.predict(img)
    feature_vector = feature_vector.reshape(1,feature_vector.shape[1])
    return feature_vector

In [ ]:
enc = encode_image("")

In [ ]:
enc.shape

In [ ]:
def predict_caption(model,image,tokenizer):
    #Add start tag for generation process
    in_text = 'start'
    max_length = 35
    #iterate over max-length of sequence
    for i in range(max_length):
        #Encode input sequences
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        #Pad the Seequence
        sequence = pad_sequences([sequence], max_length)
        #predict next word 
        yhat = model.predict([image,sequence],verbose = 0)
        #Get index with High Probability
        yhat = np.argmax(yhat)
        #Convert index to word
        word = idx_to_word(yhat,tokenizer)
        if word is None:
            break
        #Append word as input for generating next word
        in_text += " "+word
        #Stop if we reach end Tag
        if word == 'end':
            break
        return in_text

In [ ]:
predict_caption(model_vgg,enc,)